In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'
1
# Hyperparameters
n_steps = 2
window_size = 5
step_size = 2
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.00005
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


[*********************100%%**********************]  1 of 1 completed
[INFO] Training initial window with Early Stopping...
[RESULT] Initial window validation MAPE: 2.51%

[INFO] Iteration 1: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.88%
[INFO] Predicted last sample of window: Actual=299.87, Pred=297.23, MAPE=0.88%

[INFO] Iteration 2: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.36%
[INFO] Predicted last sample of window: Actual=300.82, Pred=299.75, MAPE=0.36%

[INFO] Iteration 3: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.97%
[INFO] Predicted last sample of window: Actual=304.68, Pred=301.73, MAPE=0.97%

[INFO] Iteration 4: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.31%
[INFO] Predicted last sample of window: Actual=311.89, Pred=304.68, MAPE=2.31%

[INFO] Iteration 5: training on new window with Early Stopping...
[RESULT] Window validat

[RESULT] Window validation MAPE: 0.04%
[INFO] Predicted last sample of window: Actual=308.78, Pred=308.92, MAPE=0.04%

[INFO] Iteration 38: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.02%
[INFO] Predicted last sample of window: Actual=307.09, Pred=307.16, MAPE=0.02%

[INFO] Iteration 39: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.11%
[INFO] Predicted last sample of window: Actual=301.92, Pred=305.28, MAPE=1.11%

[INFO] Iteration 40: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.09%
[INFO] Predicted last sample of window: Actual=307.92, Pred=304.57, MAPE=1.09%

[INFO] Iteration 41: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.03%
[INFO] Predicted last sample of window: Actual=305.00, Pred=304.91, MAPE=0.03%

[INFO] Iteration 42: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.73%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 2.79%
[INFO] Predicted last sample of window: Actual=358.86, Pred=348.83, MAPE=2.79%

[INFO] Iteration 83: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.11%
[INFO] Predicted last sample of window: Actual=346.57, Pred=346.19, MAPE=0.11%

[INFO] Iteration 84: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.01%
[INFO] Predicted last sample of window: Actual=346.43, Pred=346.46, MAPE=0.01%

[INFO] Iteration 85: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.11%
[INFO] Predicted last sample of window: Actual=339.13, Pred=339.50, MAPE=0.11%

[INFO] Iteration 86: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.23%
[INFO] Predicted last sample of window: Actual=343.91, Pred=343.11, MAPE=0.23%

[INFO] Iteration 87: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.25%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 0.22%
[INFO] Predicted last sample of window: Actual=369.94, Pred=369.12, MAPE=0.22%

[INFO] Iteration 127: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.81%
[INFO] Predicted last sample of window: Actual=364.20, Pred=367.15, MAPE=0.81%

[INFO] Iteration 128: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.23%
[INFO] Predicted last sample of window: Actual=362.35, Pred=363.17, MAPE=0.23%

[INFO] Iteration 129: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.14%
[INFO] Predicted last sample of window: Actual=354.47, Pred=362.07, MAPE=2.14%

[INFO] Iteration 130: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.30%
[INFO] Predicted last sample of window: Actual=362.80, Pred=361.73, MAPE=0.30%

[INFO] Iteration 131: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.00%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 1.02%
[INFO] Predicted last sample of window: Actual=383.96, Pred=380.03, MAPE=1.02%

[INFO] Iteration 171: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.93%
[INFO] Predicted last sample of window: Actual=379.66, Pred=383.20, MAPE=0.93%

[INFO] Iteration 172: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.43%
[INFO] Predicted last sample of window: Actual=380.72, Pred=382.36, MAPE=0.43%

[INFO] Iteration 173: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.43%
[INFO] Predicted last sample of window: Actual=385.38, Pred=383.72, MAPE=0.43%

[INFO] Iteration 174: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.10%
[INFO] Predicted last sample of window: Actual=387.50, Pred=387.13, MAPE=0.10%

[INFO] Iteration 175: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.88%
[INFO] Predicted last samp

...

[RESULT] Window validation MAPE: 4.28%
[INFO] Predicted last sample of window: Actual=487.61, Pred=466.76, MAPE=4.28%

[INFO] Iteration 303: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.01%
[INFO] Predicted last sample of window: Actual=472.47, Pred=481.94, MAPE=2.01%

[INFO] Iteration 304: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.99%
[INFO] Predicted last sample of window: Actual=467.99, Pred=472.60, MAPE=0.99%

[INFO] Iteration 305: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.88%
[INFO] Predicted last sample of window: Actual=463.31, Pred=472.00, MAPE=1.88%

[INFO] Iteration 306: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.85%
[INFO] Predicted last sample of window: Actual=454.80, Pred=458.65, MAPE=0.85%

[INFO] Iteration 307: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.72%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 2.71%
[INFO] Predicted last sample of window: Actual=468.15, Pred=480.86, MAPE=2.71%

[INFO] Iteration 347: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.70%
[INFO] Predicted last sample of window: Actual=472.02, Pred=468.73, MAPE=0.70%

[INFO] Iteration 348: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.01%
[INFO] Predicted last sample of window: Actual=467.99, Pred=468.03, MAPE=0.01%

[INFO] Iteration 349: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.30%
[INFO] Predicted last sample of window: Actual=464.17, Pred=465.57, MAPE=0.30%

[INFO] Iteration 350: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.53%
[INFO] Predicted last sample of window: Actual=471.43, Pred=468.95, MAPE=0.53%

[INFO] Iteration 351: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.32%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.00%
[INFO] Predicted last sample of window: Actual=503.22, Pred=503.24, MAPE=0.00%

[INFO] Iteration 391: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.14%
[INFO] Predicted last sample of window: Actual=504.10, Pred=504.80, MAPE=0.14%

[INFO] Iteration 392: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.71%
[INFO] Predicted last sample of window: Actual=501.74, Pred=505.32, MAPE=0.71%

[INFO] Iteration 393: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.62%
[INFO] Predicted last sample of window: Actual=493.14, Pred=501.13, MAPE=1.62%

[INFO] Iteration 394: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.46%
[INFO] Predicted last sample of window: Actual=487.03, Pred=489.25, MAPE=0.46%

[INFO] Iteration 395: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.68%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.24%
[INFO] Predicted last sample of window: Actual=524.20, Pred=522.92, MAPE=0.24%

[INFO] Iteration 435: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.12%
[INFO] Predicted last sample of window: Actual=523.42, Pred=524.07, MAPE=0.12%

[INFO] Iteration 436: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.78%
[INFO] Predicted last sample of window: Actual=532.80, Pred=536.95, MAPE=0.78%

[INFO] Iteration 437: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.21%
[INFO] Predicted last sample of window: Actual=538.38, Pred=531.88, MAPE=1.21%

[INFO] Iteration 438: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.28%
[INFO] Predicted last sample of window: Actual=541.13, Pred=539.61, MAPE=0.28%

[INFO] Iteration 439: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.63%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.74%
[INFO] Predicted last sample of window: Actual=547.60, Pred=551.65, MAPE=0.74%

[INFO] Iteration 479: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.43%
[INFO] Predicted last sample of window: Actual=554.88, Pred=546.95, MAPE=1.43%

[INFO] Iteration 480: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.60%
[INFO] Predicted last sample of window: Actual=556.80, Pred=553.46, MAPE=0.60%

[INFO] Iteration 481: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.85%
[INFO] Predicted last sample of window: Actual=560.90, Pred=556.13, MAPE=0.85%

[INFO] Iteration 482: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.88%
[INFO] Predicted last sample of window: Actual=571.27, Pred=566.24, MAPE=0.88%

[INFO] Iteration 483: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.85%
[INFO] Predicted last samp